Use this utlity to update the returns and std_dev fields within investment-options.csv

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import brownbear as bb

# format price data
pd.options.display.float_format = '{:0.2f}'.format

%matplotlib inline

/home/fja0568/src/brownbear
/home/fja0568/src/brownbear/universe/etf-galaxy


In [3]:
# set size of inline plots
'''note: rcParams can't be in same cell as import matplotlib
   or %matplotlib inline
   
   %matplotlib notebook: will lead to interactive plots embedded within
   the notebook, you can zoom and resize the figure
   
   %matplotlib inline: only draw static images in the notebook
'''
plt.rcParams["figure.figsize"] = (10, 7)

Globals

In [4]:
# set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True

In [5]:
# read in investment options input csv
etfs = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
etfs = list(etfs['Investment Option'])

In [6]:
# eliminate repeat symbols
symbols = set(etfs)

In [7]:
# get the timeseries for the symbols and compile into a single csv
bb.fetch_timeseries(symbols, refresh=refresh_timeseries)
bb.compile_timeseries(symbols)

......................................................................


In [8]:
# read symbols timeseries into a dataframe
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)

In [9]:
annual_returns = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns['SPY']

20.826285940457435

In [10]:
# calculate annualized returns
annual_returns_1mo = bb.annualize_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualize_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualize_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualize_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualize_returns(df, timeperiod='daily', years=5)

In [11]:
# calculate volatility
daily_returns = df.pct_change()
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)
vola['SPY']

0.1263342686281806

In [12]:
# calculate downside volatility
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)
ds_vola['SPY']

0.056264271902764874

In [13]:
# resample df on a monthly basis
df.index = pd.to_datetime(df.index)
monthly = df.resample('M').ffill()
bb.print_full(monthly['SPY'])

Date
2015-01-31   178.18
2015-02-28   188.20
2015-03-31   185.24
2015-04-30   187.06
2015-05-31   189.47
2015-06-30   185.62
2015-07-31   189.81
2015-08-31   178.24
2015-09-30   173.70
2015-10-31   188.47
2015-11-30   189.16
2015-12-31   185.89
2016-01-31   176.64
2016-02-29   176.49
2016-03-31   188.36
2016-04-30   189.10
2016-05-31   192.32
2016-06-30   192.99
2016-07-31   200.03
2016-08-31   200.27
2016-09-30   200.28
2016-10-31   196.81
2016-11-30   204.06
2016-12-31   208.19
2017-01-31   211.92
2017-02-28   220.25
2017-03-31   220.52
2017-04-30   222.71
2017-05-31   225.85
2017-06-30   227.29
2017-07-31   231.96
2017-08-31   232.64
2017-09-30   237.33
2017-10-31   242.92
2017-11-30   250.35
2017-12-31   253.38
2018-01-31   267.66
2018-02-28   257.93
2018-03-31   250.86
2018-04-30   252.16
2018-05-31   258.29
2018-06-30   259.77
2018-07-31   269.40
2018-08-31   277.99
2018-09-30   279.65
2018-10-31   260.32
2018-11-30   265.15
2018-12-31   241.81
2019-01-31   261.17
2019-02-28   26

In [14]:
# calculate monthly returns
monthly_returns = monthly.pct_change()
monthly_returns['SPY']

Date
2015-01-31     nan
2015-02-28    0.06
2015-03-31   -0.02
2015-04-30    0.01
2015-05-31    0.01
              ... 
2020-08-31    0.07
2020-09-30   -0.04
2020-10-31   -0.02
2020-11-30    0.11
2020-12-31    0.02
Freq: M, Name: SPY, Length: 72, dtype: float64

In [15]:
# calculate standard deviation
std_dev = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev['SPY']

0.18363275195421486

In [16]:
# read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]
#lines

In [17]:
# for each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev
out = []
for line in lines:
    # copy empty and comment lines
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # split line by comma delimiter; strip the fields
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # copy header
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd = std_dev[symbol]*100

    out.append('{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
               .format(fields[0], fields[1], fields[2],
                       ret_1mo, ret_3mo, ret_1yr, ret_3yr, ret_5yr, _vola, _ds_vola, sd)) 

In [18]:
# write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')